# Simple Example

This is a simple example to get familiar with how to use permchain. permchain is a pub-sub framework which makes it easy to coordinate multiple LLM actors (whether these be agents or single LLM calls). This notebook goes over a simple example of three actors:

- a writer, responsible for writing the first draft
- a editor, responsible for critiquing a written draft
- a reviser, responsible for taking a draft and associated critiques and editing it

We will first define these actors individually, and then we will show how to coordinate them such that for a given input the writer will write a draft, and then the editor and reviser will go back and forth until the editor thinks its good enough.

In [1]:
from operator import itemgetter

from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.runnables.openai_functions import OpenAIFunctionsRouter

from permchain.connection_inmemory import InMemoryPubSubConnection
from permchain.pubsub import PubSub
from permchain.topic import Topic

## Drafter

In [2]:
drafter_prompt = (
    SystemMessagePromptTemplate.from_template(
        "You are an expert on turtles, who likes to write in pirate-speak. You have been tasked by your editor with drafting a 100-word article answering the following question."
    )
    + "Question:\n\n{question}"
)
drafter_llm = ChatOpenAI(model="gpt-3.5-turbo")
drafter = drafter_prompt | drafter_llm | StrOutputParser()

In [3]:
drafter.invoke({"question": "what is art?"})

"Arrr, me hearties! What be art, ye ask? Art be a fine treasure crafted by the hands of a creative soul. It be a form o' expression, a way to share the beauty and wonders o' the world. It be a splash o' colors on a canvas, a melody playin' in yer ear, or a tale spun with words. Art be a look into the depths o' the human spirit, a glimpse into the mysteries o' life. So, me mateys, let yer hearts be filled with art, for it be the treasure that brings joy and meaning to our pirate lives!"

## Critiquer

In [4]:
editor_prompt = (
    SystemMessagePromptTemplate.from_template(
        "You are an editor. You have been tasked with editing the following draft, which was written by a non-expert. Please accept the draft if it is good enough to publish, or send it for revision, along with your notes to guide the revision."
    )
    + "Draft:\n\n{draft}"
)
editor_llm = ChatOpenAI(model="gpt-4")
functions = [
    {
        "name": "revise",
        "description": "Sends the draft for revision",
        "parameters": {
            "type": "object",
            "properties": {
                "notes": {
                    "type": "string",
                    "description": "The editor's notes to guide the revision.",
                },
            },
        },
    },
    {
        "name": "accept",
        "description": "Accepts the draft",
        "parameters": {
            "type": "object",
            "properties": {"ready": {"const": True}},
        },
    },
]
editor = editor_prompt | editor_llm.bind(functions=functions)

In [5]:
editor.invoke({"draft": "hi!"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'revise', 'arguments': '{\n  "notes": "The current draft is too short and lacks any context or detailed information. Please provide a more comprehensive and detailed draft for review."\n}'}}, example=False)

## Reviser

In [6]:
reviser_prompt = (
    SystemMessagePromptTemplate.from_template(
        "You are an expert on turtles. You have been tasked by your editor with revising the following draft, which was written by a non-expert. You may follow the editor's notes or not, as you see fit."
    )
    + "Draft:\n\n{draft}"
    + "Editor's notes:\n\n{notes}"
)
reviser_llm = ChatOpenAI(model="gpt-3.5-turbo")
reviser = reviser_prompt | reviser_llm | StrOutputParser()

In [7]:
reviser.invoke({"draft": "hi!", "notes": "too short"})

'Revised draft:\n\nHello there!'

## Hooking it all up

We can now hook it all up. This means:

1. Each chain should subscribe to some events. This can be the `input` event, or they can listen for pushes to an inbox
2. Each chain should do something with the output. This can involving returning a final answer, or pushing to an inbox

In [8]:
# create topics
editor_inbox = Topic("editor_inbox")
reviser_inbox = Topic("reviser_inbox")

In [9]:
draft_chain = (
    # Listed in inputs
    Topic.IN.subscribe()
    | {"draft": drafter}
    # The draft always goes to the editors inbox
    | editor_inbox.publish()
)

In [10]:
editor_chain = (
    # Listen for events in the editors inbox
    editor_inbox.subscribe()
    | editor
    # Depending on the output, different things should happen
    | OpenAIFunctionsRouter(
        {
            # If revise is chosen, we send a push to the revisor's inbox
            "revise": (
                {
                    "notes": itemgetter("notes"),
                    "draft": editor_inbox.current() | itemgetter("draft"),
                    "question": Topic.IN.current() | itemgetter("question"),
                }
                | reviser_inbox.publish()
            ),
            # If accepted, then we return
            "accept": editor_inbox.current() | Topic.OUT.publish(),
        },
    )
)

In [11]:
reviser_chain = (
    # Listen for events in the reviser's inbox
    reviser_inbox.subscribe()
    | {"draft": reviser}
    # Publish to the editors inbox
    | editor_inbox.publish()
)

In [12]:
web_researcher = PubSub(
    processes=(draft_chain, editor_chain, reviser_chain),
    connection=InMemoryPubSubConnection(),
)

In [13]:
import langchain

langchain.verbose = True

In [14]:
web_researcher.invoke({"question": "What food do turtles eat?"})

[{'draft': 'Turtles have specific dietary preferences that vary depending on their species. Sea turtles, for example, primarily consume seaweed, jellyfish, and occasionally fish. On the other hand, land turtles, such as tortoises, mainly graze on grass, flowers, and leafy greens. Some turtles even enjoy fruits like berries and melons in addition to their plant-based diet. Insects also make for a crunchy treat that some turtles may indulge in. Therefore, whether they inhabit land or sea, turtles have a diverse range of food options to keep their bodies nourished and satisfied.'}]

In [15]:
[*web_researcher.batch([{"question": "What food do turtles eat?"}, {"question": "Where do bears live?"}])]

[[{'draft': 'Turtles are fascinating creatures with a diverse appetite. Depending on their species and habitat, turtles consume a variety of foods. Some turtles primarily eat plants such as seaweed, grass, and algae. For instance, the green sea turtle is known to graze on seagrass beds and algae. Other turtles, like the snapping turtle, have a more carnivorous diet, feasting on insects, fish, and small crustaceans. Additionally, there are land-dwelling turtles that enjoy fruits and vegetables in their diet. For example, the box turtle has been observed eating berries and leafy greens. With such a varied diet, turtles keep their bellies full and maintain their overall health.'}],
 [{'draft': 'Revised draft:\n\nHello, readers! You may be wondering where bears live. Well, bears are known to inhabit a wide range of lands, from the icy regions of the Arctic to the lush forests of the jungles. They can be found in North America, Europe, Asia, and even some parts of South America. Bears are h